# Consume Azure Machine Learning Online Endpoints from Fabric

Azure Machine Learning provides the capability of deploying models to Online Endpoints that can then be queries from multiple clients, including Fabric. Those endpoints may host any model users created in Azure Machine Learning, or even models from the Model Catalog. Since those models are running on Online Endpoints, they don't consume Fabric's capacity and they allow you to consume any type of compute SKU that the model needs.

In this example we would see how to use MLflow to connect to Azure Machine Learning and run predictions using Endpoints instead of models.

## Prerequisites

You would need to install Azure Machine Learning plug-in for MLflow to connect to Azure Machine Learning:

In [ ]:
%pip install azureml-mlflow --quiet

StatementMeta(, , , Waiting, )


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /nfs4/pyenv-b62e6f5c-26af-4529-aaf1-b7de4f431e4f/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import pandas as pd
import mlflow

StatementMeta(, , , Waiting, )

## Get the deployment client for Azure Machine Learning workspaces

The deployment client allows you to connect to Azure Machine Learning Endpoints and Deployments, invoke endpoints and consume predictions. Use the tracking URI from the Azure portal to connect to the workspace. This is the same URI that you use for tracking and model registries:

In [ ]:
from mlflow.deployments import get_deploy_client

deployment_client = get_deploy_client("azureml://<region>.api.azureml.ms/mlflow/v1.0/subscriptions/<sub>/resourceGroups/<resource-group>/providers/Microsoft.MachineLearningServices/workspaces/<workspace>")

StatementMeta(, , , Waiting, )

Once you have a client created, use it to get access to any endpoint you have in your workspace:

## Creating an endpoint and deployment from Fabric

If you don't have an endpoint to connect to, you can still use the deployment client to create endpoints and deployments, all without installing the Azure Machine Learning SDK. To learn how to use MLflow to create endpoints, deployment, or update traffic see: [Deploy MLflow models to online endpoints [MLflow SDK]](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?view=azureml-api-2&tabs=mlflow)

## Running predict on endpoints using MLflow

Let's use the deployment client to see if we have an endpoint with the following name created:

In [ ]:
endpoint_name = "heart-classifier-qwert"
deployment_client.get_endpoint(endpoint_name)

StatementMeta(, , , Waiting, )

{'id': '/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/resourceGroups/fasantia/providers/Microsoft.MachineLearningServices/workspaces/azureml-personal-ws/onlineEndpoints/heart-classifier-qwert',
 'name': 'heart-classifier-qwert',
 'type': 'Microsoft.MachineLearningServices/workspaces/onlineEndpoints',
 'systemData': {'createdBy': 'Facundo Santiago',
  'createdAt': '2023-07-28T00:43:19.095565Z',
  'lastModifiedAt': '2023-07-28T00:43:19.095565Z'},
 'tags': {},
 'location': 'westus2',
 'identity': {'principalId': 'aec30d29-abca-4c43-8a22-d1c74dd7fff9',
  'tenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
  'type': 'SystemAssigned'},
 'kind': 'Managed',
 'properties': {'authMode': 'Key',
  'properties': {'azureml.onlineendpointid': '/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/resourcegroups/fasantia/providers/microsoft.machinelearningservices/workspaces/azureml-personal-ws/onlineendpoints/heart-classifier-qwert',
   'AzureAsyncOperationUri': 'https://management.azure.com/subs

You can also list deployments, and check their configuration.

In [ ]:
deployment_client.list_deployments(endpoint=endpoint_name)

### Scoring data using the endpoint

Let's bring some data in from Fabric OneLake to run the model on:

In [ ]:
df = spark.read.format("csv").option("header","true").load("Files/datasets/heart-disease-uci-unlabeled/heart.csv")
display(df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 24d1cd4e-1212-4677-b3cf-95d3b29d7ad9)

MLflow requires the data to be in Pandas format. So let's convert it:

In [ ]:
data = df.toPandas()

StatementMeta(, , , Waiting, )

Once, we have the data in pandas data format, we can call predict:

In [ ]:
deployment_client.predict(endpoint=endpoint_name, df=data)

StatementMeta(, , , Waiting, )

,0
0,0
1,1
2,0
3,0
4,0
...,...
298,0
299,1
300,1
301,1
